In [1]:
import jutil
import numpy as np
from collections import Counter
import pyfpgrowth

In [30]:
sql = "select t.Course_code as course_code from train_plan_course t,train_plan_credit c where t.Course_big_type_id=c.sid \
and t.Grade='2014' and c.type_name like '%专业%'  and t.Speciality_code='0408'"
# code_s = [d['course_code'] for d in jutil.load_data(sql)]
d1 = jutil.load_pd_df(sql)
code_s = d1['course_code'].tolist()

In [31]:
len(set(code_s))

53

In [4]:
sql = "select student_id,course_name,course_code,pmark,mark,term_order from view_stu_course_mark where speciality_code='0408' and grade='2014'"
df = jutil.load_pd_df(sql)
df.head()

,course_code,course_name,mark,pmark,student_id,term_order
0,0475012,VisualBasic程序设计,9,85.0,20122639,3
1,3100312,高级语言程序设计实验,7,80.0,20122639,1
2,3200510,新技术讲座,6,75.0,20122639,3
3,3232400,计算机图形学,4,70.0,20122639,4
4,3243120,计算机组成实验C,10,96.0,20122639,3


In [5]:
# 选出了与专业相关的课程
dd = df[df['course_code'].isin(code_s)]
dd = dd.drop_duplicates(subset=['student_id','course_code'])

## 按学生group,然后将其课程分数作为维度,学生数作为数据量

In [6]:
code = dd.groupby(['student_id'],as_index=False)['course_code'].apply(list)
mark = dd.groupby(['student_id'],as_index=False)['pmark'].apply(list)

In [7]:
d_mark = mark.to_dict()
d_code = code.to_dict()

In [8]:
transac = []
for k,cd in d_code.items():
    mk = d_mark.get(k,[])
#     l = len(mk) if len(mk)<20 else 20
    arr = []
    for i in range(len(mk)):
#     for i in range(l):
        if mk[i]>=90:
            arr.append(cd[i]+'_2')
#         elif mk[i]>'4':
#             arr.append(cd[i]+'_1')
        elif mk[i]<65:
            arr.append(cd[i]+"_0")
#     arr = [cd[i]+'_'+mk[i] for i in range(len(mk))]
    transac.append(arr)

In [38]:
for t in transac:
    print(t[:10])

['6010500_2', '3243120_2', '3243510_2', '0471046_2', '3143300_2', '6011310_2', '0474011_2', '3232100_2', '3243700_2', '0474014_2']
['3232100_0', '3243020_0', '0473031_0', '3232400_0', '6011310_0', '6011320_0', '6111120_0', '6111310_0', '0471071_0', '3244152_0']
['6111310_0', '6111320_0', '9990501_2', '3244238_0', '0471024_0', '3244152_0', '3045600_0', '3243020_0', '6024000_0', '3232400_0']
['6011310_2', '3243781_2']
['3243020_0', '6041943_0', '3243510_0', '6033100_0', '0471075_0', '3244152_0', '0474014_2', '3100312_0', '3143300_0']
['0471071_2', '0474014_2', '6100131_2', '0471076_2', '3100312_0', '3200510_2', '3045600_2', '0474019_2', '0474020_2', '0471046_2']
['0471054_2', '6033100_2', '9990501_2', '0474011_2', '0474019_2', '0474020_2', '6010500_2', '6011310_2', '0474014_2', '3143300_2']
['0474014_2', '3243020_0', '6041943_0', '0474011_2', '0471075_0']
['6010500_0', '6033100_0', '3200557_0', '3243781_0', '6011310_0']
['3244152_0', '0474014_2', '3045600_2', '3100312_0', '3232100_0', '0

In [9]:
hold = len(d_mark)*0.1
hold

12.0

In [10]:
p = pyfpgrowth.find_frequent_patterns(transactions=transac,support_threshold=hold)

In [11]:
rules = pyfpgrowth.generate_association_rules(p,0.7)
rules

{('0471004_2', '0474011_2'): (('0474014_2', '3045600_2'), 0.8571428571428571),
 ('0471004_2', '0474011_2', '0474014_2'): (('3045600_2',), 0.9230769230769231),
 ('0471004_2', '0474011_2', '3045600_2'): (('0474014_2',), 0.9230769230769231),
 ('0471004_2', '0474014_2'): (('0474011_2', '3045600_2'), 0.8),
 ('0471004_2', '0474014_2', '3045600_2'): (('0474011_2',), 0.9230769230769231),
 ('0471004_2', '0474020_2'): (('0474014_2',), 1.0),
 ('0471004_2', '3143300_2'): (('0474014_2',), 1.0),
 ('0471046_2', '0471071_2'): (('0474014_2',), 1.0),
 ('0471046_2', '0471072_2'): (('0474014_2',), 1.0),
 ('0471046_2', '0474011_2'): (('0474014_2', '3045600_2'), 0.8666666666666667),
 ('0471046_2', '0474011_2', '0474014_2'): (('3045600_2',), 0.8666666666666667),
 ('0471046_2', '0474011_2', '0474020_2'): (('0474014_2',), 1.0),
 ('0471046_2', '0474011_2', '3045600_2'): (('0474014_2',), 1.0),
 ('0471046_2', '0474011_2', '3143300_2'): (('0474014_2',), 1.0),
 ('0471046_2', '0474014_2'): (('3045600_2',), 0.75),
 (

## 将规则变成界面所需结果
nodes ,links

In [12]:
# 得到课程名和课程代码的关联,加上term_order,这里去重思想,按term_order最小的保留
cnts = dd[['course_code','course_name','term_order']].sort_values(by='term_order').drop_duplicates(subset=['course_code','course_name'],keep='first')
cnts

,course_code,course_name,term_order
636,3100312,高级语言程序设计实验,0
134,3200557,计算机导论A,0
136,3243781,高级语言程序设计,0
16,6011310,高等数学Ⅰ,0
226,6010500,线性代数B,0
657,6111310,大学物理CⅠ,1
656,6111120,大学物理实验Ⅱ,1
655,6033100,离散数学A,1
217,0471005,面向对象程序设计实验,1
128,0471004,面向对象程序设计,1


In [13]:
names = cnts['course_name'].tolist()
codes = cnts['course_code'].tolist()
term = cnts['term_order'].tolist()
d_course_names = dict()
for i in range(len(codes)):
    d_course_names[codes[i]] = {'name':names[i],'term':term[i]}
d_course_names

{'0471004': {'name': '面向对象程序设计', 'term': 1},
 '0471005': {'name': '面向对象程序设计实验', 'term': 1},
 '0471024': {'name': '计算机网络工程实验', 'term': 3},
 '0471046': {'name': 'J2EE体系结构及程序设计', 'term': 4},
 '0471054': {'name': '程序语言综合课程设计', 'term': 2},
 '0471071': {'name': '数据库原理', 'term': 3},
 '0471072': {'name': '数据库原理实验', 'term': 3},
 '0471075': {'name': '软件工程概论', 'term': 3},
 '0471076': {'name': '人机交互技术', 'term': 4},
 '0473031': {'name': '计算机组成原理B', 'term': 3},
 '0473041': {'name': '软件系统分析与设计', 'term': 2},
 '0473082': {'name': '数据库技术与应用', 'term': 4},
 '0474003': {'name': '计算机图形学实验', 'term': 3},
 '0474011': {'name': '数据结构实验', 'term': 2},
 '0474012': {'name': '操作系统实验', 'term': 3},
 '0474014': {'name': 'Java程序设计实验', 'term': 3},
 '0474019': {'name': '人机交互技术实验', 'term': 4},
 '0474020': {'name': '数据库技术与应用实验', 'term': 4},
 '3045600': {'name': '数字电子技术B', 'term': 2},
 '3100312': {'name': '高级语言程序设计实验', 'term': 0},
 '3143300': {'name': '数字电子技术实验B(电类)', 'term': 1},
 '3200510': {'name': '新技术讲座', 'term': 3},
 '32

In [14]:
# 把rules翻译成课程名看看

read_rule = []
for k,v in rules.items():
    d = dict()
    for post in v[0]:
        post_item = d_course_names[post[:post.find('_')]]
        post_item['type'] = '差' if post[post.find('_')+1:]=='0' else '好'
        d['post'] = post_item
        pres = []
        for pre in k:
            pre_item = d_course_names[pre[:pre.find('_')]]
            if pre_item['term']<post_item['term']:
                continue
            pre_item['type'] = '差' if pre[pre.find('_')+1:]=='0' else '好'
            pres.append(pre_item)
        if len(pres)==0:
            continue
        d['pre'] = pres
        d['prob'] = v[1]
        read_rule.append(d)
read_rule                                  

[{'post': {'name': '数字电子技术B', 'term': 2, 'type': '好'},
  'pre': [{'name': '数据结构A', 'term': 2, 'type': '差'}],
  'prob': 0.9230769230769231},
 {'post': {'name': '数字电子技术B', 'term': 2, 'type': '好'},
  'pre': [{'name': 'Java程序设计实验', 'term': 3, 'type': '好'},
   {'name': '数据结构A', 'term': 2, 'type': '差'}],
  'prob': 0.9230769230769231},
 {'post': {'name': 'Java程序设计实验', 'term': 3, 'type': '好'},
  'pre': [{'name': '新技术讲座', 'term': 3, 'type': '好'}],
  'prob': 1.0},
 {'post': {'name': 'Java程序设计实验', 'term': 3, 'type': '好'},
  'pre': [{'name': '微机与接口技术实验', 'term': 3, 'type': '好'}],
  'prob': 0.9230769230769231},
 {'post': {'name': 'Java程序设计实验', 'term': 3, 'type': '好'},
  'pre': [{'name': '微机与接口技术实验', 'term': 3, 'type': '好'}],
  'prob': 0.9230769230769231},
 {'post': {'name': 'Java程序设计实验', 'term': 3, 'type': '好'},
  'pre': [{'name': '微机与接口技术实验', 'term': 3, 'type': '好'}],
  'prob': 1.0},
 {'post': {'name': '数据结构实验', 'term': 2, 'type': '好'},
  'pre': [{'name': 'Java程序设计实验', 'term': 3, 'type': '好'},
   

In [19]:
# 转化成可读形式,给用户看
rule_str = []
s_d = set() # 去重
for r in read_rule:
    s = []
    post = r['post']['name']+' '+r['post']['type']
    pre = ''
    for p in r['pre']:
        pre += p['name']+' '+p['type']
    if post+pre not in s_d:
        s.append(pre)
        s_d.add(post+pre)
        s.append("==>")
        s.append(post)
        s.append('的可信度为: '+str(r['prob']))
        print(" ".join(s))
        rule_str.append(s)

数据结构A 差 ==> 数字电子技术B 好 的可信度为: 0.9230769230769231
Java程序设计实验 好数据结构A 差 ==> 数字电子技术B 好 的可信度为: 0.9230769230769231
新技术讲座 好 ==> Java程序设计实验 好 的可信度为: 1.0
微机与接口技术实验 好 ==> Java程序设计实验 好 的可信度为: 0.9230769230769231
Java程序设计实验 好微机与接口技术实验 好 ==> 数据结构实验 好 的可信度为: 0.9230769230769231
软件设计实习 好 ==> Java程序设计实验 好 的可信度为: 0.9230769230769231
Java程序设计实验 好软件设计实习 好 ==> 程序语言综合课程设计 好 的可信度为: 0.9230769230769231
Java程序设计实验 好Java程序设计 好 ==> 数字电子技术B 好 的可信度为: 0.8571428571428571
数据库原理实验 好 ==> Java程序设计实验 好 的可信度为: 1.0
Java程序设计实验 好 ==> 数字电子技术B 好 的可信度为: 0.8
数据库技术与应用实验 好 ==> Java程序设计实验 好 的可信度为: 1.0
数据结构实验 好 ==> 数字电子技术B 好 的可信度为: 0.8571428571428571
数据结构实验 好Java程序设计实验 好 ==> 数字电子技术B 好 的可信度为: 0.9230769230769231
Java程序设计实验 好数字电子技术B 好 ==> 数据结构实验 好 的可信度为: 0.9230769230769231
J2EE体系结构及程序设计 好数据库原理 好 ==> Java程序设计实验 好 的可信度为: 1.0
数据库原理 好Java程序设计实验 好 ==> 数字电子技术B 好 的可信度为: 0.7647058823529411
数据库原理 好计算机图形学实验 好 ==> Java程序设计实验 好 的可信度为: 1.0
数据库原理 好数据库原理实验 好 ==> Java程序设计实验 好 的可信度为: 1.0
数据库原理 好 ==> Java程序设计实验 好 的可信度为: 1.0
数据库原理 好Java程序设计实验 好数据库技术与应用实验 好 =

In [21]:
# 写入文件


In [27]:
# 将read_rule变成nodes links结构
d_size = dict()
links = []
for r in read_rule:
    d = dict()
    post = r['post']
    for pre in r['pre']:
        prob = r['prob'] if pre['type']==post['type'] else 0.1*r['prob']
        d_size[pre['name']] = d_size.get(pre['name'],0) + prob
        d_size[post['name']] = d_size.get(post['name'],0) + prob
        d['source'] = pre['name']
        d['target'] = post['name']
        links.append(d)
nodes = []
for k,p in d_size.items():
    d = dict()
    d['category'] = 0
    d['name'] = k
    d['symbolSize'] = p
    nodes.append(d)

In [28]:
links

[{'source': '数据结构A', 'target': '数字电子技术B'},
 {'source': '数据结构A', 'target': '数字电子技术B'},
 {'source': '数据结构A', 'target': '数字电子技术B'},
 {'source': '新技术讲座', 'target': 'Java程序设计实验'},
 {'source': '微机与接口技术实验', 'target': 'Java程序设计实验'},
 {'source': '微机与接口技术实验', 'target': 'Java程序设计实验'},
 {'source': '微机与接口技术实验', 'target': 'Java程序设计实验'},
 {'source': '微机与接口技术实验', 'target': '数据结构实验'},
 {'source': '微机与接口技术实验', 'target': '数据结构实验'},
 {'source': '软件设计实习', 'target': 'Java程序设计实验'},
 {'source': '软件设计实习', 'target': '程序语言综合课程设计'},
 {'source': '软件设计实习', 'target': '程序语言综合课程设计'},
 {'source': 'Java程序设计', 'target': '数字电子技术B'},
 {'source': 'Java程序设计', 'target': '数字电子技术B'},
 {'source': '数据库原理实验', 'target': 'Java程序设计实验'},
 {'source': 'Java程序设计实验', 'target': '数字电子技术B'},
 {'source': 'Java程序设计实验', 'target': '数字电子技术B'},
 {'source': '数据库技术与应用实验', 'target': 'Java程序设计实验'},
 {'source': '数据结构实验', 'target': '数字电子技术B'},
 {'source': 'Java程序设计实验', 'target': '数字电子技术B'},
 {'source': 'Java程序设计实验', 'target': '数字电子技术B'},
 {'source': '数字

In [29]:
nodes

[{'category': 0, 'name': '数据结构A', 'symbolSize': 11.344752718901326},
 {'category': 0, 'name': '数字电子技术B', 'symbolSize': 107.1993408294492},
 {'category': 0, 'name': 'Java程序设计实验', 'symbolSize': 179.3084179020076},
 {'category': 0, 'name': '新技术讲座', 'symbolSize': 1.0},
 {'category': 0, 'name': '微机与接口技术实验', 'symbolSize': 3.769230769230769},
 {'category': 0, 'name': '数据结构实验', 'symbolSize': 78.37518395347342},
 {'category': 0, 'name': '软件设计实习', 'symbolSize': 1.8461538461538463},
 {'category': 0, 'name': '程序语言综合课程设计', 'symbolSize': 16.738716984769617},
 {'category': 0, 'name': 'Java程序设计', 'symbolSize': 0.8571428571428571},
 {'category': 0, 'name': '数据库原理实验', 'symbolSize': 20.53953818369453},
 {'category': 0, 'name': '数据库技术与应用实验', 'symbolSize': 32.354416773457025},
 {'category': 0, 'name': 'J2EE体系结构及程序设计', 'symbolSize': 17.450000000000003},
 {'category': 0, 'name': '数据库原理', 'symbolSize': 14.586554621848743},
 {'category': 0, 'name': '计算机图形学实验', 'symbolSize': 13.007142857142856},
 {'category': 0